In [113]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [114]:
todate = datetime.datetime.now().strftime("%d-%m")

confirmed = pd.read_csv(f"../Data/confirmed_{todate}.csv")
deaths = pd.read_csv(f"../Data/deaths_{todate}.csv")
recovered = pd.read_csv(f"../Data/recovered_{todate}.csv")

In [115]:
confirmed.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,584,609,634,663,678,712,726,736,750,766


In [116]:
deaths.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,36,36,40,42,43,47,50,57,58,60
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,26,26,27,27,27,27,28,28,30,30


In [117]:
recovered.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,135,150,166,179,188,188,207,220,228,252
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,327,345,356,385,394,403,410,422,431,455


## Drop 'Lat' & 'Long' , SUM & GROUP BY countries

In [118]:
def summed(df):
    summed = df.drop(["Lat", "Long"], axis=1).groupby("Country/Region").sum()
    return (summed)

In [119]:
confirmed = summed(confirmed)

In [120]:
deaths = summed(deaths)

In [121]:
recovered = summed(recovered)

## Dates in Var

In [122]:
today = datetime.date.today()
yesterday = (today - datetime.timedelta(days=1)).strftime("%m/%d/%y")
yesterday

'04/29/20'

In [123]:
def is_one_digit_month(str):
    if str[0] == '0':
        str_old = str
        str = ''
        for i in str_old[1:]:
            str += i
        return str
    else:
        return str
yesterday = is_one_digit_month(yesterday)

## Display Dates in rows

In [124]:
confirmed.head(2)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939
Albania,0,0,0,0,0,0,0,0,0,0,...,584,609,634,663,678,712,726,736,750,766


In [125]:
confirmed_melt = pd.melt(confirmed.reset_index(),
                         id_vars = "Country/Region",
                         value_vars = confirmed.columns[1:],
                         var_name = "Date",
                         value_name = "Confirmed")
confirmed_melt.head()

,Country/Region,Date,Confirmed
0,Afghanistan,1/23/20,0
1,Albania,1/23/20,0
2,Algeria,1/23/20,0
3,Andorra,1/23/20,0
4,Angola,1/23/20,0


## Israel total confirmed for yesterday

In [126]:
mask_israel = confirmed_melt["Country/Region"] == "Israel"
israel = confirmed_melt[mask_israel].groupby("Country/Region")["Confirmed"].max().to_frame()
israel

,Confirmed
Country/Region,
Israel,15834


## Top 5 countries total confirmed

In [127]:
top_5_countries = confirmed_melt.groupby("Country/Region")["Confirmed"].max()\
.to_frame().sort_values("Confirmed", ascending=False).head()
top_5_countries

,Confirmed
Country/Region,
US,1039909
Spain,236899
Italy,203591
France,169053
United Kingdom,166441


## How many more confirmed in top 5 than Israel

In [128]:
top_5_countries["diff_israel"] = top_5_countries["Confirmed"] - israel["Confirmed"][0]
top_5_countries[["diff_israel"]]

,diff_israel
Country/Region,
US,1024075
Spain,221065
Italy,187757
France,153219
United Kingdom,150607


# Date first revealed for each country

In [129]:
mask = confirmed_melt["Confirmed"] > 0
first_date_discover = confirmed_melt[mask].groupby("Country/Region")["Date"].min().to_frame().sort_values("Date")
first_date_discover

,Date
Country/Region,
China,1/23/20
Vietnam,1/23/20
US,1/23/20
Thailand,1/23/20
Taiwan*,1/23/20
...,...
Sao Tome and Principe,4/10/20
Western Sahara,4/10/20
Yemen,4/10/20


# Date first revealed for each country + how many

In [130]:
fc = first_date_discover.merge(confirmed_melt,
                               how="inner",
                               left_on=["Country/Region", "Date"],
                               right_on=["Country/Region", "Date"])
fc.sort_values("Date")

,Country/Region,Date,Confirmed
0,China,1/23/20,643
1,Vietnam,1/23/20,2
2,US,1/23/20,1
3,Thailand,1/23/20,3
4,Taiwan*,1/23/20,1
...,...,...,...
182,Yemen,4/10/20,1
183,Malawi,4/10/20,9
180,Sao Tome and Principe,4/10/20,4
181,Western Sahara,4/10/20,4


In [34]:
# How many plus each day
# Add Israel to the bold plot
# Make colors same in every plot (per country)
# plot only last month
# Maybe Remove China because they are boring now